In [ ]:
import pandas as pd
import torch
import yaml

from model.crecs import CRECS
from utils.misc import send_to_device

In [ ]:
items = pd.read_csv("data/ml-20m/movies.csv")[["item_id", "item_title"]]

num_items = items["item_id"].nunique()

# Initialize models and load weights
args = yaml.safe_load(open("configs/collaborative.yaml", "r"))

device = "cuda" if torch.cuda.is_available() else "cpu"

args["model"]["recommender"]["num_items"] = num_items 

model = CRECS(weights="weights/collaborative/crecs.pt", **args["model"]).to(device)

In [ ]:
features = (torch.tensor([num_items, 0]), torch.tensor([5.0, 3.5]))

features = send_to_device(features, device)

request = "I want to watch a popular romantic comedy"

item_ids, item_ratings = send_to_device(model.predict(features, request, k=10), "cpu")


# Create response message
recommended_items = list(items.iloc[item_ids]["item_title"])[:5]

response_message = f"Here are some movie recommendations for you: {', '.join(recommended_items)}"